In [ ]:
!wget -O Aquarium_Data.zip "https://app.roboflow.com/ds/nDIuacDgZx?key=GZKyX0cGHs"

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('Aquarium_Data.zip') as target_file:
  target_file.extractall('Aquarium_Data')

In [ ]:
!cat /content/Aquarium_Data/data.yaml

In [ ]:
!pip install PyYAML

In [ ]:
import yaml

data = {
    'train' : '/content/Aquarium_Data/train/images/',
    'val' : '/content/Aquarium_Data/valid/images',
    'test' : '/content/Aquarium_Data/test/images',
    'names' : ['follicle'],
    'nc' : 1
}

with open('Aquarium_Data/data.yaml', 'w') as f:
  yaml.dump(data, f)


with open('Aquarium_Data/data.yaml', 'r') as f:
  aquarium_yaml = yaml.safe_load(f)
  display(aquarium_yaml)

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics

ultralytics.checks()

In [ ]:
from ultralytics import YOLO

model = YOLO('follicle-ai.pt')

In [ ]:
print(type(model.names), len(model.names))

print(model.names)

In [ ]:
model.train(data='/content/Aquarium_Data/data.yaml', epochs=10, imgsz=640)

In [ ]:
# 모델 학습 후 결과를 평가하기
results = model.val()

# results_dict에서 정확도 정보 가져오기
metrics = results.results_dict

# 출력
print(f"Precision: {metrics['metrics/precision(B)']:.4f}")
print(f"Recall: {metrics['metrics/recall(B)']:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import cv2

# 이미지 경로 설정
image_path = '/content/gilt—9.jpg'

results = model(image_path)  # 모델 예측 수행

# 첫 번째 예측 결과에서 'show' 메서드 호출
results[0].show()  # 예측된 이미지를 보여줍니다

# 예시로, 이미지와 예측 결과를 Matplotlib으로 표시하려면:
image = cv2.imread(image_path)
# YOLO 모델의 예측 결과로 바운딩 박스를 이미지에 추가
for *box, conf, cls in results.xywh[0]:  # 모델의 예측 결과
    x1, y1, x2, y2 = map(int, box)  # 바운딩 박스 좌표 추출
    label = results.names[int(cls)]  # 예측된 라벨 이름
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # 바운딩 박스 그리기
    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)  # 라벨 텍스트 추가

# 이미지 표시
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')  # 축을 제거
plt.show()